# Install

In [ ]:
!pip install pyserini
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114.8 MB 76 kB/s 
     |████████████████████████████████| 13.5 MB 38.9 MB/s 
     |████████████████████████████████| 5.3 MB 35.2 MB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 1.3 MB 43.7 MB/s 
     |████████████████████████████████| 4.9 MB 42.8 MB/s 
     |████████████████████████████████| 2.0 MB 25.6 MB/s 
     |████████████████████████████████| 188 kB 47.4 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 7.6 MB 29.5 MB/s 
     |████████████████████████████████| 163 kB 69.7 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, htt

# Mount Drive and change path

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Introduction-to-Information-Retrieval/HW/3')

Mounted at /content/drive


# Text Preprocessing

In [ ]:
import json
from json.decoder import JSONDecodeError

dic = { "\n": "", "\t": "", ".": "", "BACKGROUND": "", "METHODS": "", "RESULTS": "", "CONCLUSIONS": "", 
        "OBJECTIVE": "", "`": "", "'": "", ",": "", ";": "", ":": "", "(": "", ")": ""}

def replace_all(text):
  for i, j in dic.items():
    text = text.replace(i, j)
  return text

def remove_whitespace(text):
    text = " ".join(text.split())
    return "".join(text.rstrip().lstrip())

data = {"id":[],"contents":[]}
first_time = True


with open('pubmed 220 train.txt') as file:
    string = ""
    for line in file:
      if "###" in line:
        line = line.replace("###", "").replace("\n", "")
        data["id"].append(line)
        if first_time:
          first_time = False
        else:
          string = remove_whitespace(string)
          data["contents"].append(string)
        string = ""
      
      else:
        string = string + replace_all(line)

string = remove_whitespace(string)
data["contents"].append(string)


json_data = []
for i in range(len(data["contents"])):
  json_data.append({"id": data["id"][i], "contents": data["contents"][i]})


with open("./json/data.json", "w") as file:
  json.dump(json_data, file, indent=2)










# Index documents

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input ./json \
  --index ./json \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2022-10-14 07:08:38,052 INFO  [main] index.IndexCollection (IndexCollection.java:248) - Setting log level to INFO
2022-10-14 07:08:38,056 INFO  [main] index.IndexCollection (IndexCollection.java:251) - Starting indexer...
2022-10-14 07:08:38,056 INFO  [main] index.IndexCollection (IndexCollection.java:252) - ============ Loading Parameters ============
2022-10-14 07:08:38,056 INFO  [main] index.IndexCollection (IndexCollection.java:253) - DocumentCollection path: ./json
2022-10-14 07:08:38,070 INFO  [main] index.IndexCollection (IndexCollection.java:254) - CollectionClass: JsonCollection
2022-10-14 07:08:38,070 INFO  [main] index.IndexCollection (IndexCollection.java:255) - Generator: DefaultLuceneDocumentGenerator
2022-10-14 07:08:38,071 INFO  [main] index.IndexCollection (IndexCollection.java:256) - Threads: 1
2022-10-14 07:08:38,071 INFO  [main] index.IndexCollection (IndexCollection.java:257) - Language: en
2022-10-14 07:08:38,072 INFO  [main] index.IndexCollection (IndexCollection

# Search documents using inverted index

In [ ]:
from pyserini.search.lucene import LuceneSearcher
import time

start = time.time()
searcher = LuceneSearcher('./json')
hits = searcher.search('cancer')
end = time.time()

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print(f"{hits[i].raw}")

print("Elapsed time :", end - start, "s")


 1 24747090 2.50920
{
  "id" : "24747090",
  "contents" : "We examine the role of body mass index in the assessment of prostate cancer risk A total of 3258 participants who underwent biopsy including 1902 men with a diagnosis of prostate cancer were identified from the Selenium and Vitamin E Cancer Prevention Trial The associations of body mass index with prostate cancer and high grade prostate cancer were examined using logistic regression adjusting for age race body mass index adjusted prostate specific antigen digital rectal examination family history of prostate cancer biopsy history prostate specific antigen velocity and time between study entry and the last biopsy The prediction models were compared with our previously developed body mass index adjusted Prostate Cancer Prevention Trial prostate cancer risk calculator Of the study subjects 491 % were overweight and 293 % were obese After adjustment among men without a known family history of prostate cancer increased body mass ind

# Search documents using brute force

In [ ]:
import re

hits = []

def Brute_force_search(term):
  for i in range(len(data["contents"])):
    if term in data["contents"][i].lower():
      hits.append({"id": data["id"][i], "contents": data["contents"][i]})
      
start = time.time()
Brute_force_search("cancer")
end = time.time()

for i in range(len(hits)):
  print(i + 1)
  print("{")
  print("  \"id\" : \"{}\",".format(hits[i]["id"]))
  print("  \"contents\" : \"{}\",".format(hits[i]["contents"]))
  print("}")

print("Elapsed time :", end - start, "s")

Streaming output truncated to the last 5000 lines.
{
  "id" : "25038542",
  "contents" : "Rectal cancer continues to be devastating malignancy worldwide Sphincter preservation is the need of the hour Distal anastomosis is more prone to leaks Proximal diversion in form of ileostomy may be used to protect distal anastomosis To compare two groups of low anterior resection with and without diversion ileostomy in rectal cancer patients A prospective hospital based study of 78 rectal carcinoma patients were taken for the study Inclusion criteria was operable rectal cancer 4-12 cm from anal verge Patients were randomized into two groups Group - A 34 patient patients with low anterior resection with ileostomy LAR with ileostomy Group - B 44 patients patients with low anterior resection without ileostomy LAR without ileostomy Quality of life was assessed by scoring done by self designed method A total score of 0-20 given for various parameters Skin excoriation was the commonest complication Sto